# Reproduce  H. Huang , et. al , Faraday Discuss., 2018, 208, 555 – 573 

This notebook demonstrates reproducing the results of the paper with [Larch](https://xraypy.github.io/xraylarch/xafs/) using the corresponding published dataset (raw data). 

Huang, Haoliang, Nassr, Abu Bakr Ahmed Amine, Celorrio, Verónica, Taylor, S. F. Rebecca, Puthiyapura, Vinod Kumar, Hardacre, Christopher, Brett, Dan J. L., Russell, Andrea E. (2018) **Effects of heat treatment atmosphere on the structure and activity of Pt3Sn nanoparticle electrocatalysts: a characterisation case study.** Faraday Discussions. V. 208. pp. 555-573. DOI: [10.1039/c7fd00221a](https://doi.org/10.1039/c7fd00221a).

- **Corresponding author**: Russell, Andrea E.
- **E-mail**: a.e.russell@soton.ac.uk

The data used for reproducing the results was published in the Southampton Instituional Repository [10.5258/SOTON/D0408](https://eprints.soton.ac.uk/421798/), with the name Xray_data.zip - Dataset.


For more details about Larch, see [Larch Website](https://xraypy.github.io/xraylarch/xafs/)

|CDI Entity  |Link       |DOI|
|:-----------|:----------|:---|
|Publication |[cdi_pub:37](http://cdi.ukcatalysishub.org/articles/37)|[10.1039/c7fd00221a](https://doi.org/10.1039/c7fd00221a) |
|Data Object |[cdi_do:537](http://cdi.ukcatalysishub.org/datasets/537)|[10.5258/SOTON/D0408](https://eprints.soton.ac.uk/421798/)|

## Reproduce Figure 4 - XANES Analysis
The first set of XAS results in the paper are in [Figure 4](https://pubs.rsc.org/image/article/2018/fd/c7fd00221a/c7fd00221a-f4_hi-res.gif)

To reproduce this image we need to:
- A: Identify the XANES data used (SnO2, Air, Ar, H2 and Sn foil)
  - Plot normalised flattened mE of the spectra on the range 29190 – 29450 
  - Plot first derivate of the spectra on the range 29190 – 29215
- B: Take Sn foil and SnO2 and do LCF of the peaks against H2, Ar, and Air
- C: Take Sn K-Edge of H2/H2 and SnO2 and do LCF of the peaks against H2, Ar, and Air

|Key|Content|
|:--|:--|
|OC|parent|
|OCA| Ar treated|
|OCH| H2 treated|
|OCO|air-treated|

In [ ]:
# Library with the functions that rplicate those provided by athena
# normalisation, merging, re-binning, LCF
# and visualisation (plotting)
import lib.manage_athena as athenamgr  

# File handling
from pathlib import Path

#plotting library
import matplotlib.pyplot as plt
# inline: shows plot in notebook
# tk: shows plot in popup
%matplotlib inline 

### Input parameters (variables)
The variables in the next cell are the processing values that indicate where to get the data from, the pattern of the files to process and the number of files to process. These can be changed to process different datasets.

### Column and group names.

Knowing the elements of the input data set it is possible to start manipulating and visualising the XAFS data. 

In this case energy and $\mu$ are provided, but the names of the columns do not match the names used by Larch provinding the custom column names. 

We rename the columns. The name of the groups is the same as the name of the file. To make things easier we can also adjust the file name.


In [ ]:
# variables that can be changed to process different datasets
data_path = "./wf_data/pub_037/XAFS_prj/Sn K-edge/ascii"

data_mappings={"PtSn":  "*_PtSn_OC_A*",
              "Air":  "*_PtSn_OCO_A*",
              "Ar":  "*_PtSn_OCA_A*",
              "H2":  "*_PtSn_OCH_A*",
              "PtSn H":  "*_PtSn_OC_H*", 
              "Air H":  "*_PtSn_OCO_H*",
              "Ar H":  "*_PtSn_OCA_H*",
              "H2 H":  "*_PtSn_OCH_H*",}

f_prefix = "PtSn_KEdge"
column_labels = "energy time I0 It Iref  mu lnItIref"

### Get data and merge results for each sample in list
The below reads the raw readings for eache of the samples in data mappings and merges them to create the signals to be processed.



In [ ]:
# start processing createa an output dir and sets the logger
source_path, out_path = athenamgr.files_setup(f_prefix, data_path)
print("Source:", source_path, "Output:", out_path)


merged_results={}
# read all samples, merge and then normalise
for a_sample in data_mappings:
    files_list = athenamgr.get_files_list(source_path, data_mappings[a_sample])

    # read the files for each sample
    sample_list = []
    for i_count, a_file  in enumerate(files_list):
        file_name = a_file.name
        f_suffix = str(i_count).zfill(4) 
        p_name = f_prefix+f_suffix
        p_path = Path(out_path , p_name + ".prj")
        a_group = athenamgr.read_text(a_file, column_labels)
        sample_list.append(a_group)
        
    # merge readings for sample
    merged_xas = athenamgr.merge_readings(sample_list)

    # rename group (same as the file name)
    merged_xas.filename = a_sample
    merged_results[a_sample] = merged_xas
    # calculate pre-edge and post edge and add them to group
    # using defaults
    xas_data = athenamgr.fit_pre_post_edge(merged_xas)

# results in the paper are recalibrated to 29204
recalibrate_e0_to = 29204

for a_sample in merged_results:
    merged_results[a_sample] = athenamgr.recalibrate_energy(merged_results[a_sample], recalibrate_e0_to)


### Get standards
The code below reads the Sn Foil and SnO2 project and retrieves the foil data used to copare to the readings


In [ ]:
# get the Sn foil from project:
sn_foil = "./wf_data/pub_037/XAFS_prj/Sn foil.prj"
# read the input file 
sn_foil_prj = athenamgr.read_project(sn_foil)

sn_foil_group = athenamgr.get_group(sn_foil_prj, 'merge')
sn_foil_group.filename = "Sn Foil"
sn_foil_group = athenamgr.recalibrate_energy(sn_foil_group, recalibrate_e0_to)
merged_results["Sn Foil"] = sn_foil_group

# get the Sn O2 standard from project:
sno2 = "./wf_data/pub_037/XAFS_prj/SnO2 0.9 2.6-13.5 gbkg.prj"
# read the input file 
sno2_prj = athenamgr.read_project(sno2)

sno2_group = athenamgr.get_group(sno2_prj, "SnO2_0_9_2_6_13_5_0_8_1_0_with_theory")#'SnO2_0_9')
sno2_group.filename = "SnO2"
sno2_group = athenamgr.recalibrate_energy(sno2_group, recalibrate_e0_to)
merged_results["SnO2"] = sno2_group

### Plot the mu signals for the reading and the Sn Foil (Figure 4A)

These are the plots shown on Fig. 4 A

In [ ]:
include_groups = ["Air", "Ar", "H2", "Sn Foil", "SnO2"]
for a_sample in merged_results:
    if merged_results[a_sample].filename in include_groups:
        plt = athenamgr.plot_normalised(merged_results[a_sample])
    
plt.xlim([29180, 29400])
plt.show()

for a_sample in merged_results:
    if merged_results[a_sample].filename in include_groups:
        plt = athenamgr.plot_derivative(merged_results[a_sample])
    
plt.xlim([29190, 29210])
plt.show()

### Linear combination fitting - LCF (Figure 4 A and B)
Lineal combination fitting with defaults for the H2, Ar and Air readings (Shown on Fig. 4 B)

In [ ]:
# import custom plot functions (replicate plots in paper)
import paper01_plots as c_plots

In [ ]:
lcf_components = [merged_results["Sn Foil"],merged_results["SnO2"]] # List of groups to use as components 

r_H2 = athenamgr.lcf_group(merged_results["H2"], lcf_components)
r_Ar = athenamgr.lcf_group(merged_results["Ar"], lcf_components)
r_Air = athenamgr.lcf_group(merged_results["Air"], lcf_components)

c_plots.compare_lcf_plot([merged_results["H2"],r_H2], [merged_results["Ar"],r_Ar], [merged_results["Air"],r_Air])
plt.show()

print(r_H2.arrayname)

In [ ]:
r_HH2 = athenamgr.lcf_group(merged_results["H2 H"], lcf_components)
r_HAr = athenamgr.lcf_group(merged_results["Ar H"], lcf_components)
r_HAir = athenamgr.lcf_group(merged_results["Air H"], lcf_components)

c_plots.compare_lcf_plot([merged_results["H2 H"],r_HH2], [merged_results["Ar H"],r_HAr], [merged_results["Air H"],r_HAir])
plt.show()

In [ ]:
print("Goodness of LCF fits")
print ("LCF for H2:\t", "%.4f" % r_H2.rfactor)
print ("LCF for Ar:\t", "%.4f" % r_Ar.rfactor)
print ("LCF for Air:\t", "%.4f" % r_Air.rfactor)
print ("LCF for H\\H2:\t","%.4f" % r_HH2.rfactor)
print ("LCF for H\\Ar:\t","%.4f" % r_HAr.rfactor)
print ("LCF for H\\Air:\t","%.4f" % r_HAir.rfactor)


## Reproduce Figure 5 XAS Processing
Rebin is required for XAS processing (see [Athena Manual](https://bruceravel.github.io/demeter/documents/Athena/process/rebin.html). So after the XANES analysis

In [ ]:
# Rebin Ar, Air and H2 Samples 
rebin_labels = ["H2", "Ar", "Air"]
rebinned_groups = {}
rebinned_gr=None
for a_sample in merged_results:
    if a_sample in rebin_labels:
        rebinned_gr = athenamgr.rebin_group(merged_results[a_sample])
        rebinned_gr.arrayname = a_sample+" Rebbined"
        rebinned_groups[a_sample+" Rebbined"] = rebinned_gr
        print(a_sample, "rebinned to" , len(rebinned_gr.energy),"from", len(merged_results["Ar"].energy))

athenamgr.save_groups([rebinned_groups["H2 Rebbined"],rebinned_groups["Ar Rebbined"],rebinned_groups["Air Rebbined"]], "rebinned.prj")

In [ ]:
# read the input file 
athena_prj = athenamgr.read_project("rebinned.prj")

athena_groups = athenamgr.get_groups(athena_prj)

group_names = []
for a_group in athena_groups:
    group_names.append(a_group.label)
    
for a_group in athena_groups:
    athenamgr.calc_with_defaults(a_group)    


### Crystal files and fitting parameters

Define the location of the crystal files to be used in fitting.
The fitting parameters can also be changed. 

In [ ]:
# Fitting input variables
# All crystals for this fit are stored at: 
#    C:/harwell/PSDI_test/wf_data/pub_037/additional/
#       1692395a.cif  Pt3Sn for Sn
#       1690711.cif   SnO2
#       1680385.cif   PtO2
#       1692395b.cif  Pt3Sn for Pt
# new test replace Pt3Sn (1692395a.cif) with PtSn
#       1692396.cif   PtSn for Sn 


crystal_files = ["./wf_data/pub_037/additional/1690711.cif",
                 "./wf_data/pub_037/additional/1692396.cif",
                 "./wf_data/pub_037/additional/1680385.cif",
                 "./wf_data/pub_037/additional/1692395b.cif",] #

gds_parms_f = "SnK_edge_gds.csv" # paths and parameters can
sel_paths_f = "SnK_edge_sp.csv"  # be saved and retrieved.

show_graph = False # False to prevent showing graphs

# variables for fit
fit_vars = {}
fit_vars['fitspace']='r'
fit_vars['kmin']=0 
fit_vars['kmax']=10
fit_vars['kw']=2 
fit_vars['dk']=1
fit_vars['window']='hanning'
fit_vars['rmin']=0.0
fit_vars['rmax']=5.0

# Need to specify the absorbing atom and maximum radius
absorbing_atoms = ["Sn","Sn","O","Pt"]
max_radius = 5.0

### Atoms and FEFF

Larch does larch does not include a means for running atoms. Atoms is needed to get input for feff and calculate paths. Currently, the fastest option is to run Artemis to obtain the input (.inp) file for feff from a crystal file ('.cif' or '.inp')

The code below shows how subprocess can be used to call perl, execute a small perl script that runs Artemis Atoms, and saves the output file ('inp') in a new directory.

The file can then be used to run FEFF from Larch to calculate scattering paths.


In [ ]:
# Library with the functions that execute 
# Atoms and FEFF to generate scattering paths
import lib.atoms_feff as feff_runner     

# Set parameters          
# library containign functions tho manage fit, at read, write 
# GDS parameters, and scattering paths. 
import lib.manage_fit as fit_manager  

from larch import Interpreter
session = Interpreter()

# run feff and get a list of feff dirs
feff_dirs = feff_runner.run_feff(crystal_files, absorbing_atoms, max_radius) 

# could prevent running if the files still exist and they are recent (<month old)

In [ ]:
# read saved parameters from input gds file
gds = fit_manager.read_gds(gds_parms_f, session)
# show gsd group parameters in a spreadsheet
this_sheet = fit_manager.show_gds(gds)
# save gsd group parameters in a csv file
fit_manager.save_gds(gds, gds_parms_f)

In [ ]:
# just in case something was changed
# read the gds data and save it to the csv file 
gds = fit_manager.spreadsheet_to_gds(this_sheet, session)
# save gsd group parameters in a csv file
fit_manager.save_gds(gds, gds_parms_f)

#### Showa Select  FEFF paths

To select a path change the value of the select column to 1 in the table displayed after running the cell below.

In [ ]:
# read saved selected paths from input file
selected_paths = fit_manager.read_selected_paths_list(sel_paths_f, session)

path_sheet = fit_manager.show_feff_paths(crystal_files, selected_paths)
display(path_sheet)

#### 2.5.2 Assing parameters to selected paths

To define the parameters enter values like those presented in the table below into the spreadsheet that appears after running the code in the following cell. The values should correspond to those defined as GDS parameters previously.

|file                      |label    | s02 |e0   |sigma2 |deltar      |
|:------------------------ |------   |-----|-----|-------|------------|
|1690711_feff/feff0001.dat | O.Sn.1	 |amp  |enot |sssno  |alpha\*reff |
|1692396_feff/feff0001.dat | Pt.Sn.1 |amp  |enot |sssnpt |alpha\*reff |
|1680385_feff/feff0001.dat | Pn.O.1  |amp  |enot |sspto  |alpha\*reff |
|1692395b_feff/feff0001.dat| Pn.Pt.1 |amp  |enot |ssptpt |alpha\*reff |
|1692395b_feff/feff0002.dat| Sn.Pt.2 |amp  |enot |ssptsn |alpha\*reff |


**Note:** Labelling is used for reference only using Artemis-FEFF given names. Larch's FEFF does not label paths.


In [ ]:
sp_sheet = fit_manager.show_selected_paths(path_sheet, selected_paths)

In [ ]:
fit_manager.save_selected_paths_list(sp_sheet, sel_paths_f)
selected_paths = fit_manager.read_selected_paths_list(sel_paths_f, session)

### 2.6 Run Fit

XAS fitting is performed in three steps:

1. Create a Transform group to hold the set of Fourier transform parameters, fitting ranges, and space in which the data and sum of paths are to be compared (R space)
2. Create a Dataset group,consistaining of the three components required for fitting(data, paths, and transform group)
3. FEFFIT is run with the list of parameters (gds) for the fit, and the dataset or list of datasets groups.

In [ ]:
# run fit
fit_list = {}
for a_group in athena_groups:
    trans, dset, out = fit_manager.run_fit(a_group, gds, selected_paths, fit_vars, session)
    fit_list[a_group.filename]=[trans, dset, out]

In [ ]:
for a_fit in fit_list:
    chikr_p = c_plots.plot_markers(fit_list[a_fit][1],fit_vars['rmin'],fit_vars['rmax'],fit_vars['kmin'],fit_vars['kmax'], a_fit)
    chikr_p.show()

In [ ]:
for a_fit in fit_list:
    print("R-factor %.2f"%(fit_list[a_fit][2].rfactor*100)+"%")

In [ ]:
for a_fit in fit_list:
    fit_report = fit_manager.get_fit_report(fit_list[a_fit][2], session)
    print(fit_report)


## Reproducibility of results

All of the results can be reproduced. however there is a small issue regarding which crystal files to use. These are not included in the published dataset. 

Pending contactiong authors for validation of process.

    